# 12. 템플릿 모듈화 하기
여기선 장고의 템플릿 확장 기능을 사용해 템플렛 요소 중 몇가지를 모듈화 해본다. 그전에 왜 모듈화를 하는것이 좋은지 알아보자
___
## 메인 영역 모듈화 하기
### 왜 템플릿 요소를 모듈화 할까?
템플릿의 메인영역을 제외한 나머지 영역은 디자인이 일관되게 유지되어야 한다. 하나의 페이지에 메인 영역을 뺀 나머지 영역을 만들어 놓고 Post_list일때 Post_detail_page 일때를 구분하여 메인영역만 채워주면 편리하다.
### post_list.html 모듈화 하기
현재 기준에서 모든 테스트를 통과한 포스트 목록 페이지를 수정해보자. 수정하고 나서 테스트하면 수정한 내용에 이상이 없는지 쉽게 확인할 수 있다.
#### (1) base.html 만들기
먼저 post_list.html을 복사하여 base.html을 만든다. base.html은 메인영역을 제외하고 모든 페이지에 일관될 공통 영역만 남기기 위한 파일이다.  
base.html 에 id='main-area' 인 요소를 모두 지우고   

{% block main_area %}   
{% endblock % }  

를 추가한다.
#### (2) base.html을 확장해 post_list.html 넣기
이제 post_list.html에는 block안에 들어가는 요소만 있으면 되므로 base.html에서 지웠던 부분만 남기고 나머지를 모두지운다.   
보기 쉽게 비유하자면 아래와 같다. 나머지 영역은 base.html 에서 가져오고 메인영역은 post_list만 남긴다.  

>(나머지 영역)={% extends 'blog/base.html %}  
{% block main-area %}  
(메인 영역)  
{% endblock %}  
(나머지 영역)  

#### (3) 수정 결과 학인하기
post_list의 함수만 테스트를 하고 싶다면  
>python manage.py test blog.tests.TestView.test_post_list  

라고 입력하면 된다. 테스느 결과가 OK가 나온다.
### post_detail.html 모듈화 하기
#### (1) base.html을 확장해 post_detail.html 넣기
post_detail.html역시 base.html을 이용할 수 있게 수정해보자. 방법은 동일하다. post_detail.html에서 메인 영역에 해당하는 부분만 남기고 다 지운후 블록을 지정하면 된다. 현재 post_detail.html의  div 태그 중 container / row / col-lg-8/안에 잇는 내용만 남기면된다.  
#### (2) 제목 블록 따로 만들기
또한 현재 title이 반영되지 않으므로   
base.html에서 타이틀을 {% block head_title %}Blog{% endblock %}로 바꿔준뒤,  post_detail.html 에서도  

{% block head_title %}  
    {{post.title}}-Blog  
{% endblock %}  

을 추가 해준다. 
#### (3) post_detail.html에서 포스트 영역과 댓글 영역 구분하기
다시 테스트를 해보면 오류가 타나나지는 않지만 post_area가 None이라고 나온다. 포스트 영역과 댓글 영역을 아직 구분하지 않았기 때문이다.  
post영역은 post-area 로 댓글은 coment-area로 감싼다음 다시 test해보면 이번에는 OK가 출력된다.


## 내비게이션 바와 푸터 모듈화하기
현재까지는 포스트 목록과 상세 페이지등으로 가기위해선 URL 주소를 직접 입력했다. 아직 navbar에 링크를 설정하지 않았기 때문이다.
### Navbar에 link 추가
#### (1) 원인
먼저 blog/templates/blog/base.html의 navbar소스 코드를 보면 href 값에 아무것도 있지 않다. 이부분을 장고의 urls.py에서 처리할 수 있도록 수정해야한다.  
#### (2) 테스트 코드 작성
navbar의 정상 유무를 확인하는 테스트 코드는 목록 검사, 상세페이지 검사 함수 모두 동일하기 들어있따. 그렇기 때문에 중복을 피하려면 따로 함수를 만들어 주자.  
test로 시작하는 이름은 TestCase가 테스트를 위한 함수라고 인식 하기 때문에 navbar_test로 명명하고 위의 함수들이 파싱한 HTML을 그대로 받아 soup을 매개변수로 사용한다.  다시 검사를 해봐도 정상적으로 OK가 나온다.
#### (3) Navbar 버튼의 href 링크 테스트 코드 만들기
네비바의 버튼을 클릭했을 때 제대로 된 링크로 연결되는지 확인하는 것이 navbar_text()함수의 역할이다. 각 버튼을 클릭했을 때 연결된 링크로 가는지 확인이 필요하다.  
navbar.find로 네비바에서 HOME 을 가진 문구를 가진 a요소를 찾아 변수에 저장, 그리고 a요소에서 href 속성을 찾아 값이 '/'인지 확인하는 것이다. 마찬가지로 Blog일경우 '/blog/'를 찾는 함수이다. 테스트를 하면 OK가 나온다.
### include로 네비바와 푸터 모듈화 하기
웹 사이트의 디자인 통일성을 위해 네비게이션바와 푸터는 모든 페이지가 동일한게 좋다. 그러므로 nav footer태그의 코드들을 따로 관리하는것이 편하다. 장고의 include를 사용하면 HTML을 요소별로 관리 호출이 가능하다.
#### (1) 네비게이션바를 navbar.html로 모듈화 하기
먼저 blog/templates/blog/폴더에 navbar.html을 새롭게 만든후 base.html의 nav 태그~ 모달관련 코드 까지 싹 잘라서 붙여넣기한다.